In [4]:
#Importing necessary libraries
import pandas as pd
import numpy as np

In [6]:
#Loading the dataset 
df = pd.read_parquet('/Users/calvinfrederick/Desktop/university/Case Compeition : Intern etc./Datathon 2024/catB_train.parquet')
print(df)

          clntnum race_desc ctrycode_desc clttype stat_flag min_occ_date  \
19550  91b546e924   Chinese     Singapore       P    ACTIVE   2017-10-31   
4600   896bae548c   Chinese     Singapore       P    ACTIVE   2007-05-23   
13337  f364439ae6    Others     Singapore       P    ACTIVE   2019-08-31   
15074  70f319cfe1   Chinese     Singapore       P    ACTIVE   2021-10-18   
19724  2647a81328   Chinese     Singapore       P    ACTIVE   2018-07-20   
...           ...       ...           ...     ...       ...          ...   
11284  3363260248      None     Singapore       P    LAPSED   2016-08-01   
11964  c3509f0aef   Chinese     Singapore       P    ACTIVE   1990-03-20   
5390   6e44cfb3e7   Chinese     Singapore       P    ACTIVE   2007-12-31   
860    dd4e362f9f    Others     Singapore       P    ACTIVE   2007-09-01   
15795  3788d06ec7    Indian     Singapore       P    ACTIVE   2011-10-24   

       cltdob_fix cltsex_fix  flg_substandard  flg_is_borderline_standard  \
19550  197

In [11]:
#Preliminary Inspection
#First few rows of the dataset
print(df.head())

          clntnum race_desc ctrycode_desc clttype stat_flag min_occ_date  \
19550  91b546e924   Chinese     Singapore       P    ACTIVE   2017-10-31   
4600   896bae548c   Chinese     Singapore       P    ACTIVE   2007-05-23   
13337  f364439ae6    Others     Singapore       P    ACTIVE   2019-08-31   
15074  70f319cfe1   Chinese     Singapore       P    ACTIVE   2021-10-18   
19724  2647a81328   Chinese     Singapore       P    ACTIVE   2018-07-20   

       cltdob_fix cltsex_fix  flg_substandard  flg_is_borderline_standard  \
19550  1974-05-09     Female              0.0                         0.0   
4600   1979-11-11       Male              0.0                         0.0   
13337  1976-01-28       Male              0.0                         0.0   
15074  1976-03-19     Female              0.0                         0.0   
19724  1995-07-31     Female              0.0                         0.0   

       ...  recency_giclaim  giclaim_cnt_success  recency_giclaim_success  \
195

In [12]:
#Summary of the dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 17992 entries, 19550 to 15795
Columns: 304 entries, clntnum to f_purchase_lh
dtypes: float64(44), int64(46), object(214)
memory usage: 41.9+ MB
None


In [13]:
#Basic statistical summary of numberical features
print(df.describe())

       flg_substandard  flg_is_borderline_standard  flg_is_revised_term  \
count     16978.000000                16978.000000         16978.000000   
mean          0.040287                    0.033102             0.000707   
std           0.196638                    0.178907             0.026577   
min           0.000000                    0.000000             0.000000   
25%           0.000000                    0.000000             0.000000   
50%           0.000000                    0.000000             0.000000   
75%           0.000000                    0.000000             0.000000   
max           1.000000                    1.000000             1.000000   

       flg_is_rental_flat  flg_has_health_claim  flg_has_life_claim  \
count        16978.000000          16978.000000        16978.000000   
mean             0.012546              0.048769            0.003593   
std              0.111306              0.215391            0.059835   
min              0.000000              0

In [14]:
#Handling missing values
print(df.isnull().sum())

#We can see that race_desc and ctrycode_desc have missing values, what do we want to do with these values?
#Numerical variables (f_purcahse_lh)
##Small dataset, impute missing numerical values with mean, median or a value that makes sense

#Categorical variables (race_desc, ctrycode_desc)
##Use predictive modelling: Train a model to predict the missing categorical values based on other available information.
##Impute with a placeholder category like "Unknown" if predictive modelling is not feasible.

#Variables that do not provide insight for customer journey experience
##flg_gi_claim_29d435_ever and its variants


clntnum                         0
race_desc                    3996
ctrycode_desc                  20
clttype                         0
stat_flag                       0
                            ...  
flg_gi_claim_29d435_ever    17992
flg_gi_claim_058815_ever    17992
flg_gi_claim_42e115_ever    17992
flg_gi_claim_856320_ever    17992
f_purchase_lh               17282
Length: 304, dtype: int64


In [15]:
#Filling the mising values for race_desc (categorical) using mode
df['ctrycode_desc'].fillna(df['ctrycode_desc'].mode()[0], inplace=True)

In [16]:
#Dropping columns that are irrelevant to the agenda
df.drop(['flg_gi_claim_29d435_ever'], axis=1, inplace=True)
df.drop(['flg_gi_claim_058815_ever'], axis=1, inplace=True)
df.drop(['flg_gi_claim_42e115_ever'], axis=1, inplace=True)
df.drop(['flg_gi_claim_856320_ever'], axis=1, inplace=True)

In [18]:
#Dealing with the race_desc
##We do not consider race a key factor in our analysis, we can consider imputing an 'Unknown' into it
df['race_desc'].fillna('Unknown', inplace=True)

In [19]:
#Removing duplicate rows
df = df.drop_duplicates()

In [22]:
#Imputing missing values for f_purchase_lh because it is an important factor in our analysis
from sklearn.impute import KNNImputer

# Create a DataFrame containing only the column you want to impute
column_to_impute = 'f_purchase_lh'
df_impute = df[[column_to_impute]].copy()

# Initialize the KNNImputer with the number of neighbors you want to consider
knn_imputer = KNNImputer(n_neighbors=5)  # You can adjust n_neighbors as needed

# Fit the imputer on the data and transform to impute missing values
df_impute[column_to_impute] = knn_imputer.fit_transform(df_impute)

# Replace the original column in the main DataFrame with the imputed values
df[column_to_impute] = df_impute[column_to_impute]

# Check if there are still missing values
print(df.isnull().sum())

clntnum                          0
race_desc                        0
ctrycode_desc                    0
clttype                          0
stat_flag                        0
                             ...  
giclaim_cnt_success          17992
recency_giclaim_success      17992
giclaim_cnt_unsuccess        17992
recency_giclaim_unsuccess    17992
f_purchase_lh                    0
Length: 300, dtype: int64


In [ ]:
#Inputing variables for success and unsuccess 
##important factors as successful and unsuccessful claims could contribute to customer disengagement or satisfaction
from sklearn.impute import KNNImputer

# Create a DataFrame containing only the columns you want to impute
columns_to_impute = ['giclaim_cnt_success', 'recency_giclaim_success', 'giclaim_cnt_unsuccess', 'recency_giclaim_unsuccess']
df_impute = df[columns_to_impute].copy()

# Initialize the KNNImputer with the number of neighbors you want to consider
knn_imputer = KNNImputer(n_neighbors=5)  # You can adjust n_neighbors as needed

# Fit the imputer on the data and transform to impute missing values
df_impute = knn_imputer.fit_transform(df_impute)

# Replace the original columns in the main DataFrame with the imputed values
df[columns_to_impute] = df_impute

In [ ]:
#Encode categorical variables using one-hot encoding and label encoding because ML models require input and output variables to be numeric

In [ ]:
#Variables that we want to use
##CATEGORICAL VARIABLES

#giclaim_cnt_success          
#recency_giclaim_success      
#giclaim_cnt_unsuccess        
#recency_giclaim_unsuccess
#Demographic and Household Info - #annual_income_est, 
#Policy and Claim History - f_ever_declined_la, 


In [28]:
#Lets do decision tree to determine best input variables
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Assuming 'df_encoded' is your DataFrame with encoded features
# Define your target variable and features
target_column = 'annual_income_est'  # Replace 'target' with the name of your target variable
X = df.drop(columns=[target_column])
y = df[target_column]

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)

# Fit the model to your training data
clf.fit(X_train, y_train)

# Get feature importances
feature_importances = clf.feature_importances_

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort features by importance (descending order)
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print or visualize the feature importances
print(feature_importance_df)


ValueError: could not convert string to float: '8547edff5c'

In [29]:
print(df)

          clntnum race_desc ctrycode_desc clttype stat_flag min_occ_date  \
19550  91b546e924   Chinese     Singapore       P    ACTIVE   2017-10-31   
4600   896bae548c   Chinese     Singapore       P    ACTIVE   2007-05-23   
13337  f364439ae6    Others     Singapore       P    ACTIVE   2019-08-31   
15074  70f319cfe1   Chinese     Singapore       P    ACTIVE   2021-10-18   
19724  2647a81328   Chinese     Singapore       P    ACTIVE   2018-07-20   
...           ...       ...           ...     ...       ...          ...   
11284  3363260248   Unknown     Singapore       P    LAPSED   2016-08-01   
11964  c3509f0aef   Chinese     Singapore       P    ACTIVE   1990-03-20   
5390   6e44cfb3e7   Chinese     Singapore       P    ACTIVE   2007-12-31   
860    dd4e362f9f    Others     Singapore       P    ACTIVE   2007-09-01   
15795  3788d06ec7    Indian     Singapore       P    ACTIVE   2011-10-24   

       cltdob_fix cltsex_fix  flg_substandard  flg_is_borderline_standard  \
19550  197

In [ ]:
#SELECTING FEATURES
#NUMERICAL
##annual_income_est
##

#CATEGORICAL
##f_purchase_lh
##f_ever_declined_la 
##
